<a href="https://colab.research.google.com/github/jackschedel/AutoCalibr/blob/main/AutoCalibr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y libassimp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libassimp-dev is already the newest version (5.2.2~ds0-1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [2]:
#!rm -r assimp
#!git clone https://github.com/assimp/assimp.git
#!cd assimp/port/PyAssimp
#!python setup.py install

In [10]:
!pip install numpy
!pip install pyassimp
!pip install bpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.4/371.4 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 68.5 MB/s eta 0:00:00


In [18]:
# mount Google Drive to VM disk
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
import pyassimp
import os
import bpy

FBX = '/content/drive/MyDrive/datasets/AutoCalibr/fbx/'
PLY = '/content/drive/MyDrive/datasets/AutoCalibr/ply/'

In [30]:
!rm -r /content/drive/MyDrive/datasets/AutoCalibr/ply
!mkdir /content/drive/MyDrive/datasets/AutoCalibr/ply

for f in os.listdir(FBX):
    if f.endswith('.fbx'):
        # Delete all mesh objects to avoid exporting multiple models into the same file
        bpy.ops.object.select_all(action='DESELECT')
        bpy.ops.object.select_by_type(type='MESH')
        bpy.ops.object.delete()

        # Load in FBX file
        bpy.ops.import_scene.fbx(filepath=os.path.join(FBX, f))

        # Select the object
        obj_object = bpy.context.selected_objects[0]
        bpy.context.view_layer.objects.active = obj_object

        # Export object to PLY
        bpy.ops.export_mesh.ply(filepath=os.path.join(PLY, f.replace('.fbx', '.ply')))

rm: cannot remove '/content/drive/MyDrive/datasets/AutoCalibr/ply': No such file or directory
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Coldheart.ply' in 0.366
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Vigilance Wing.ply' in 0.165
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/RatKing.ply' in 0.409
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Riskrunner.ply' in 0.574
FBX version: 7400
Export completed "/content/drive/MyDrive/datasets/AutoCalibr/ply/Skyburner's oath.ply" in 0.158
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Prometheus Lens.ply' in 0.470
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Tractor Cannon.ply' in 1.171
FBX version: 7400
Export completed '/content/drive/MyDrive/datasets/AutoCalibr/ply/Wardcliff Coil.ply' in 0.844
FBX version: 7400
Expor